## Iterations over ML process steps




1. Provided data was loaded and shuffled.
2. The data set was analyzed by looking at correlation heatmap and scatter plots between AimoScore and the features.
3. The data set was then split into training and evaluation subsets (70% - 30%)
4. Some model was chosen, then it was trained on training subset and  evaluated on evaluation subset. Table below show which models were tested and their evaluation.
5. Step number 4 was repeated several times, each time using a different model.
6. Model that produced the best results was implemented in the system.


## Design & coding rules

1. Follow Python code convention. Run PEP8 test before commit.
2. Follow the SOLID principles
3. Make the merge request from the feature branch to the development, hold the hold review for the MR.
4. Design the foundation architecture that would be flexible to new changes in each new sprint.
5. Microservice architecture
6. Unit testing
7. Write the required packages and their versions to run the project
8. Implement the docker file to make the project independent of the hardware.

## DevOps & REST

1. Implemented Exception handling for the score evaluation end-point
2. Added static IP to the server (http://ec2-13-48-176-197.eu-north-1.compute.amazonaws.com:5000/)

# Tested models, reasoning behind choosing these models and test results

### Sprint 3

Exclusion of some columns in feature groups (FMS, NASM, time or joins of them) did not provide an accurate enough model in Sprint 2, therefore other approaches were used for this sprint. In addition, more metrics were to look at how good the model is - mean absolute error(MAE) and residual sum of squares(RSS). Both of these values have to be as small as possible.

1. Backwards elimitation - a feature group is selected, then the feature that performs the worst is removed (feature with p value > 0.05 and whichever has the highest p value). The results are of models that had all p values < 0.05.

|             | FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|
| R2  | 0.42 | 0.43 | 0.54 | 0.54 |
| MSE | 0.03 | 0.03 | 0.03 | 0.03 |
| MAE | 0.14 | 0.14 | 0.13 | 0.12 |
| RSS | 0.03 | 0.03 | 0.03 | 0.03
|features selected | 7 | 7 | 10 | 11

The correlation map was checked again as doing backwards elimitation indicated that some features in groups might be dependant on each other. And indeed, some were. Any correlation between a single group's features that was higher than 0.5 was considered to be high correlation.

2. Using interaction terms on highly correlated features by multiplying these features together and not removing them.

|             | FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|
| R2  | 0.42 | 0.43 | 0.559 | 0.61 |
| MSE | 0.03 | 0.03 | 0.02 | 0.02 |
| MAE | 0.14 | 0.14 | 0.12 | 0.12 |
| RSS | 0.03 | 0.03 | 0.02 | 0.02

3. Using interaction terms on highly correlated features by multiplying these features together and then removing them.

|             | FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|
| R2  | 0.4 | 0.41 | 0.550 | 0.561 |
| MSE | 0.03 | 0.03 | 0.02 | 0.02 |
| MAE | 0.14 | 0.14 | 0.12 | 0.12 |
| RSS | 0.03 | 0.03 | 0.02 | 0.02

4. There are lots of features and applying various interaction terms on them gets difficult. On top of that, the data is scattered and noisy. It is difficult to see which kind of model to use as the data seem to be random and each feature on its own does not have high enough correlation. It is clear, though, that the relationship between features and the label is not linear. So to see if there will be any improvement, non-linear transformations were applied to each feature in a feature group (for this test, each feature was raised to the power of 2 as well as to the power of 1).

|             | FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|
| R2  | 0.454 | 0.46 | 0.65 | 0.656 |
| MSE | 0.03 | 0.03 | 0.02 | 0.02 |
| MAE | 0.14 | 0.14 | 0.11 | 0.11 |
| RSS | 0.03 | 0.03 | 0.02 | 0.02

5. For this experiment, each feature was raised to the powers up to 5 (so power of 1, power of 2 etc. until power of 5). This slowed down model training and prediction by quite a bit.

|             | FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|
| R2  | 0.51 | 0.521 | 0.7 | 0.71 |
| MSE | 0.03 | 0.03 | 0.02 | 0.02 |
| MAE | 0.14 | 0.14 | 0.11 | 0.11 |
| RSS | 0.03 | 0.03 | 0.02 | 0.02

6. Finally, all feature groups were selected and each feature was raised to the power of 2 again.

|             | FMS + NASM            | FMS + NASM + time         |
|:------------------:|:------------------:|:------------------:|
| R2  | 0.68 | 0.68 |
| MSE | 0.02 | 0.02 |
| MAE | 0.11 | 0.11 |
| RSS | 0.02 | 0.02 |

While the results might look promising at the first glance as R-squared has improved the most in the last test, it seems that other metrics barely improved when raising features by higher power. 

Model that includes powers 1 and 2 from feature groups FMS, NASM and time (test 6) was chosen as it has given the best results for this sprint, even if it is overfitting. 

The reason why transformations of powers up to 5 was not chosen was because its prediction is visibly slower than the rest of the models' and it does not show any improvement in any other metrics besides R-squared.


### Sprint 2

As the given dataset is split into three types of data - FMS, NASM and time - it was decided to first simply select some combinations of these three groups and see which model would perform the best. Model accuracy was defined by low MSE and high R2 value.

1. Removing lower colerration feature (to AimoScore) from symmetrical pairs (if pair is at positions [4 6] and column at position 4 has lower correlation to AimoScore than column at position 6, 4 gets removed)

| FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|
| MSE: 0.03, R2: 0.49| MSE: 0.03, R2: 0.41 | MSE: 0.03, R2: 0.53 | MSE: 0.03, R2:0.56 |

2. For this test, FMS and NASM group features were combined (and time as well, eventually). Then, symmetrical pairs of some feature group (the group that was chosen is italicized in the results) had their correlation to checked against AimoScore. Those that have higher correlation of the two in the pair were selected. This is similar to test 4, but several feature groups are used now.

| >*FMS* + NASM         | FMS + >*NASM*       | >*FMS* + >*NASM*     | >*FMS* + NASM + time | FMS + >*NASM* + time | >*FMS* + >*NASM* + time |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|:-------------:|
| MSE: 0.02, R2: 0.58| MSE: 0.02, R2: 0.57 | MSE: 0.02, R2: 0.55 | **MSE: 0.02, R2:0.58** | **MSE: 0.02, R2:0.58** | MSE: 0.02, R2:0.56 |

3. Removing higher colerration feature (to AimoScoire) from symmetrical pairs (same idea as previous step, but this time higher correlation feature gets removed)

| FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|
| MSE: 0.04, R2: 0.31| MSE: 0.04, R2: 0.32| MSE: 0.03, R2: 0.49 | MSE: 0.03, R2:0.48 |

4. Just like in experiment 5, FMS and NASM group features were combined, together with time later on. Then symmetrical pairs of some selected feature group had their correlation tested against AimoScore individually and the one that had lower correlation was selected. This is similar to test 6, but now several feature groups are combined.

| >*FMS* + NASM         | FMS + >*NASM*       | >*FMS* + >*NASM*     | >*FMS* + NASM + time | FMS + >*NASM* + time | >*FMS* + >*NASM* + time |
|:------------------:|:------------------:|:-------------------:|:-------------:|:-------------:|:-------------:|
| MSE: 0.02, R2: 0.56| MSE: 0.02, R2: 0.56 | MSE: 0.03, R2: 0.49 | MSE: 0.02, R2:0.57 | MSE: 0.03, R2:0.57 | MSE: 0.03, R2:0.52 |

5. Removing left value of symmetrical pairs

| FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|
| MSE: 0.04, R2: 0.34| MSE: 0.04, R2: 0.39| MSE: 0.03, R2: 0.49 | MSE: 0.03, R2: 0.53 |

6. Removing right value of symmetrical pairs

| FMS                | FMS + time         | NASM                | NASM + time   |
|:------------------:|:------------------:|:-------------------:|:-------------:|
| MSE: 0.04, R2: 0.35| MSE: 0.04, R2: 0.32| MSE: 0.03, R2: 0.47 | MSE: 0.03, R2:0.49 |

7. Selecting whole separate feature groups (so, for example, only FSM, or only NASM)


| FMS                | FMS + time         | NASM                | NASM + time   | time               |
|:------------------:|:------------------:|:-------------------:|:-------------:|:------------------:|
| MSE: 0.03, R2: 0.42| MSE: 0.03, R2: 0.44| MSE: 0.03, R2: 0.56 | MSE: 0.02, R2:0.55 | MSE: 0.05, R2: 0.09 |

8. Selecting both, FMS and NASM

| With time          | Without time       |
|:------------------:|:------------------:|
|**MSE: 0.02, R2: 0.60**| **MSE: 0.02, R2: 0.57**|

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.metrics as metrics
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
data = pd.read_csv('data.csv', decimal=',')
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
def poly(arr, n):
    # Generates a string for smf.ols() with a polynomial
    # of a certain degree given some DataFrame of feature names
    features = '+'.join(arr)
    for x in range(n+1):
        if x==0: continue
        power = ''
        for f in arr:
            power = power + '+np.power({0}, {1})'.format(f, x)
        features = features + power
    return features


In [ ]:
# Split training and validation set 70%-30%
train_set, validation_set = train_test_split(data, test_size=0.3)

In [ ]:
# Select all features groups (FMS, NASM and time)
selected_features = poly(data.iloc[:, 1:41], 2)
reg = smf.ols('AimoScore ~ ' + selected_features, data=train_set).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              AimoScore   R-squared:                       0.686
Model:                            OLS   Adj. R-squared:                  0.670
Method:                 Least Squares   F-statistic:                     43.51
Date:                Sat, 22 Feb 2020   Prob (F-statistic):          2.35e-298
Time:                        22:29:46   Log-Likelihood:                 877.17
No. Observations:                1465   AIC:                            -1612.
Df Residuals:                    1394   BIC:                            -1237.
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              2.1251      0.622      3.419      0.001       0.906       3.344
No_1_Angle_Deviation                  -0.0091      0.053     -0.172      0.864      -0.113       0.095
No_2_Angle_Deviation                  -0.0142      0.062     -0.230      0.818      -0.135       0.107
No_3_Angle_Deviation                  -0.1699      0.139     -1.225      0.221      -0.442       0.102
No_4_Angle_Deviation                   0.1222      0.080      1.525      0.127      -0.035       0.279
No_5_Angle_Deviation                  -0.0371      0.046     -0.799      0.425      -0.128       0.054
No_6_Angle_Deviation                   0.0725      0.070      1.042      0.297      -0.064       0.209
No_7_Angle_Deviation                   0.0461      0.051      0.907      0.365      -0.054       0.146
No_8_Angle_Deviation                   0.3856      0.336      1.149      0.251      -0.273       1.044
No_9_Angle_Deviation                   0.0004      0.075      0.005      0.996      -0.147       0.148
No_10_Angle_Deviation                 -0.1226      0.063     -1.959      0.050      -0.245       0.000
No_11_Angle_Deviation                 -1.5251      0.597     -2.556      0.011      -2.696      -0.355
No_12_Angle_Deviation                  0.0002      0.060      0.004      0.997      -0.118       0.119
No_13_Angle_Deviation                  0.0955      0.059      1.606      0.109      -0.021       0.212
No_1_NASM_Deviation                   -0.0371      0.046     -0.799      0.425      -0.128       0.054
No_2_NASM_Deviation                    0.0461      0.051      0.907      0.365      -0.054       0.146
No_3_NASM_Deviation                   -0.0091      0.053     -0.172      0.864      -0.113       0.095
No_4_NASM_Deviation                   -0.1226      0.063     -1.959      0.050      -0.245       0.000
No_5_NASM_Deviation                    0.0955      0.059      1.606      0.109      -0.021       0.212
No_6_NASM_Deviation                    0.7104      0.254      2.796      0.005       0.212       1.209
No_7_NASM_Deviation                    0.2257      0.074      3.042      0.002       0.080       0.371
No_8_NASM_Deviation                   -0.2732      0.193     -1.414      0.157      -0.652       0.106
No_9_NASM_Deviation                   -0.0618      0.135     -0.459      0.647      -0.326       0.203
No_10_NASM_Deviation                   0.2698      0.110      2.444      0.015       0.053       0.486
No_11_NASM_Deviation                  -0.1151      0.154     -0.748      0.455      -0.417       0.187
No_12_NASM_Deviation                   1.7478      0.177      9.864      0.000       1.400       2.095
No_13_NASM_Deviation                  -0.6267      0.410     -1.529      0.126      -1.431      

In [ ]:
smf_pred = reg.predict(validation_set)
mse = mean_squared_error(validation_set.iloc[:, 0], smf_pred)
mae = metrics.mean_absolute_error(validation_set.iloc[:, 0], smf_pred)
rss = np.mean((smf_pred - validation_set.iloc[:, 0]) ** 2)
print('Mean squared error: %.2f' % mse)
print('Mean absolute error: %.2f' % mae)
print('Residual sum of squares: %.2f' % rss)

Mean squared error: 0.02
Mean absolute error: 0.11
Residual sum of squares: 0.02


In [ ]:
# Split training and validation set 80%-20%
train_set, validation_set = train_test_split(data, test_size=0.2)

In [ ]:
# Select all features groups (FMS, NASM and time)
selected_features = poly(data.iloc[:, 1:41], 2)
reg = smf.ols('AimoScore ~ ' + selected_features, data=train_set).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              AimoScore   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     51.17
Date:                Sat, 22 Feb 2020   Prob (F-statistic):               0.00
Time:                        22:32:07   Log-Likelihood:                 1012.0
No. Observations:                1675   AIC:                            -1882.
Df Residuals:                    1604   BIC:                            -1497.
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              2.0581      0.577      3.568      0.000       0.927       3.189
No_1_Angle_Deviation                   0.0597      0.050      1.196      0.232      -0.038       0.158
No_2_Angle_Deviation                  -0.0564      0.057     -0.987      0.324      -0.169       0.056
No_3_Angle_Deviation                  -0.2913      0.130     -2.234      0.026      -0.547      -0.036
No_4_Angle_Deviation                   0.1531      0.074      2.058      0.040       0.007       0.299
No_5_Angle_Deviation                   0.0163      0.042      0.388      0.698      -0.066       0.099
No_6_Angle_Deviation                   0.0782      0.065      1.209      0.227      -0.049       0.205
No_7_Angle_Deviation                  -0.0105      0.047     -0.227      0.821      -0.102       0.081
No_8_Angle_Deviation                  -0.0080      0.298     -0.027      0.979      -0.592       0.576
No_9_Angle_Deviation                   0.0023      0.070      0.034      0.973      -0.134       0.139
No_10_Angle_Deviation                 -0.0590      0.059     -1.001      0.317      -0.175       0.057
No_11_Angle_Deviation                 -1.5367      0.561     -2.738      0.006      -2.638      -0.436
No_12_Angle_Deviation                 -0.1150      0.056     -2.047      0.041      -0.225      -0.005
No_13_Angle_Deviation                  0.0635      0.056      1.129      0.259      -0.047       0.174
No_1_NASM_Deviation                    0.0163      0.042      0.388      0.698      -0.066       0.099
No_2_NASM_Deviation                   -0.0105      0.047     -0.227      0.821      -0.102       0.081
No_3_NASM_Deviation                    0.0597      0.050      1.196      0.232      -0.038       0.158
No_4_NASM_Deviation                   -0.0590      0.059     -1.001      0.317      -0.175       0.057
No_5_NASM_Deviation                    0.0635      0.056      1.129      0.259      -0.047       0.174
No_6_NASM_Deviation                    0.7647      0.235      3.261      0.001       0.305       1.225
No_7_NASM_Deviation                    0.1979      0.069      2.866      0.004       0.062       0.333
No_8_NASM_Deviation                   -0.2398      0.175     -1.367      0.172      -0.584       0.104
No_9_NASM_Deviation                   -0.1430      0.124     -1.153      0.249      -0.386       0.100
No_10_NASM_Deviation                   0.4163      0.103      4.028      0.000       0.214       0.619
No_11_NASM_Deviation                  -0.1709      0.142     -1.202      0.230      -0.450       0.108
No_12_NASM_Deviation                   1.8940      0.163     11.602      0.000       1.574       2.214
No_13_NASM_Deviation                  -0.3913      0.361     -1.083      0.279      -1.100      

In [ ]:
smf_pred = reg.predict(validation_set)
mse = mean_squared_error(validation_set.iloc[:, 0], smf_pred)
mae = metrics.mean_absolute_error(validation_set.iloc[:, 0], smf_pred)
rss = np.mean((smf_pred - validation_set.iloc[:, 0]) ** 2)
print('Mean squared error: %.2f' % mse)
print('Mean absolute error: %.2f' % mae)
print('Residual sum of squares: %.2f' % rss)

Mean squared error: 0.02
Mean absolute error: 0.11
Residual sum of squares: 0.02


## **Architecture and Design**

![Architecture](https://i.imgur.com/Nvf4xcK.jpg)

**ML core**  
This part includes the steps for the robust machine learning pipeline:  
•	Data preprocessing – module to process the raw data before feeding it to the model  
•	Model selection – the architecture is designed in a way to make it easy to replace the already implemented model with the new one. This will boost the speed of conducting the experiments and makes it more probable to get a more accurate system as the end goal.  
•	Model training – is defined with the common interface so that the parameters can be easily replaced for the new experiments.  
•	Model estimator – the module that loads the module, keeps it at the background in RAM and makes the predictions. Using this approach one can increase the speed of model inference since the model is loaded only once in a server.  

**REST API**  
It serves as the wrapper for the score predicting model and as the connector to the front-end part. It was decided to implement Flask based REST API as it is fast in development, can be iteratively updated and saves time for the experiment part of project, which is scalable for low-mid sized projects.

**Front End**  
Includes the interface and functions so that the user can interact with the system. Includes the following pages:   
1. Home page – serves as the intro to the product site.  
2. Evaluation – takes the user’s document which includes the features to describe the process of conducting the exercise by the person. As the output, the user receives the expert score between 0 and 1, which indicates how accurately the technique was executed by the customer. 0 – Worst score, 1 – best score.  

**Config**  
Config serves as a separate file that includes all the parameters needed for the system to operate properly. It can be changed to change the behavior of the system.  

**Deployment**  
The project was wrapped using the docker container in order to make the system independent from the hardware on which it was developed and make it scalable. The AWS server with minimum hardware specifications was chosen for Sprint #1 as it is optimal for this version of the product and.


## **Devops process**

<h3>Libraries</h3>

**easydict==1.9**  
EasyDict allows to access dict values as attributes (works recursively). A Javascript-like properties dot notation for python dicts.  

**pandas==0.25.1**  
pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with structured (tabular, multidimensional, potentially heterogeneous) and time series data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python.

**numpy==1.18.1**  
NumPy is the fundamental package for scientific computing with Python. It contains among other things:
•	a powerful N-dimensional array object  
•	sophisticated (broadcasting) functions  
•	tools for integrating C/C++ and Fortran code  
•	useful linear algebra, Fourier transform, and random number capabilities  
•	scikit-learn==0.22.1  

**pathlib==1.0.1**  
pathlib offers a set of classes to handle filesystem paths. It offers the following advantages over using string objects:  
•	No more cumbersome use of os and os.path functions. Everything can be done easily through operators, attribute accesses, and method calls.  
•	Embodies the semantics of different path types. For example, comparing Windows paths ignores casing.  
•	Well-defined semantics, eliminating any warts or ambiguities (forward vs. backward slashes, etc.).

**flask==1.1.1**  
Flask is a lightweight WSGI web application framework. It is designed to make getting started quick and easy, with the ability to scale up to complex applications.


## **User's roadmap**

![Flowchart](https://i.imgur.com/Oo5xhso.jpg)


## **Design rules**
The following design rules have been set up and will be followed throughout the entire project.

**1. Small classes:** 
- For readability reasons small classes are implemented.

**2. Single responsibility:**
- For classes and methods single responsibility is implemented. The reason for this is to keep the classes and methods focused on a single concern, which results in making the class or method more robust.

**3. High cohesion:**
- Achieved by keeping parts of a code base that are related to each other in a single place.

**2. Low coupling:**
- Achieved by seperating unrelated parts of the code base as much as possible.

## **Coding rules** 
The following style guide for python code has been set up in order to set certain coding rules which will be followed throughout the project.

<h3>pep8</h3>

**1. Indentation:**  
- 4 spaces are used per indentation level.
- continuation lines align wrapped elements vertically (using Python's implicit joining inside bracekts or parantheses) or using hanging indents.

**2. Spaces vs Tabs:**
- The preferred indentation method is Spaces.

**3. Maximum Line Length:** 
- All lines limited to a maximum of 79 characters.
- wrap long lines by using Python's implied line continuation inside parantheses, brackets and braces. By wrapping expressions in parantheses long lines can be broken over multiple lines.

**4. Blank Lines:**
- Top-level function and class definitions are surrounded with two blank lines
- Method definitions are surrounded by a single blank line (method definitions inside a class).

**5. Whitespaces:**
- Surround binary operators with a single whitespace on both sides.
- Never: trailing whitespaces; more than one space around assignments or operators; immediately before the opening parenthesis / comma and after closing paranthesis; immediately inside parentheses / brackets / braces.

**6. Imports:**
- All imports are on separate lines.
- Put at the top of the file, before module globals and constants.

**7. Comments:**
- Inline comments are separated by two spaces from the code and start with a # together with a single space.
- Never: obvious or contradicting comments; commented out code.

**8. Naming Convention:**
- Intention-revealing and pronounceable names
- Nouns for class names
- Verbs for method names  
</br>
</br>

<h3>Setup of automated formatting in PyCharm for simple rules:</h3>

**1. General autoformat (like spaces):**
- Windows: ctrl + alt + l
- macOS: Alt + ⌘ + L

**2. Configuration edit to break long lines:**  
- file -> settings -> editor -> python:  
- wrapping and braces: Hard Wrap at 79  
- wrapping and braces: ensure right margin is not exceeded [x]  

Now, with ctrl + alt + l  long lines will be formatted automatically and running pep8 ml-project-first or pep8 app.py will give no warnings.

Server URL: http://ec2-13-48-176-197.eu-north-1.compute.amazonaws.com:5000/